In [1]:
import pandas as pd

In [3]:
trips = pd.read_csv("T:/DCProjects/StoryMap/BikeCounting/BikeShare/Data/trips_org_dst.csv")

In [4]:
trips.head()

,RouteID,BikeID,UserID,Location,Latitude,Longitude,Date,Time,OriginDestination
0,10551719,19607,1197171,13th & Kincaid,44.045876666666665,-123.078595,2019-01-01,00:30,Origin
1,10551732,19812,1404147,13th & Kincaid,44.04576,-123.0786,2019-01-01,00:34,Origin
2,10551790,19749,1210268,Matthew Knight Arena,44.045471666666664,-123.06529333333333,2019-01-01,01:07,Origin
3,10551796,19743,1210268,Matthew Knight Arena,44.045426666666664,-123.065345,2019-01-01,01:10,Origin
4,10551856,19597,557417,Broadway & Willamette,44.049645,-123.092835,2019-01-01,01:50,Origin


In [38]:
trips.Location = trips.Location.str.replace(', ', ' @ ', regex=True)

In [39]:
trips.Location = trips.Location.str.replace('Monroe St & Blair Blvd ', 'Monroe St & Blair Blvd', regex=True)

In [5]:
import geopandas as gpd

In [6]:
stations = gpd.read_file(r"T:\DCProjects\StoryMap\BikeCounting\BikeMap\data\mapdata\bike_share_stations.shp")

In [7]:
stations.head()

,station id,name,geometry
0,hub_3828,Eugene Family YMCA,POINT (-13701591.712 5471253.204)
1,hub_3829,Monroe St & Blair Blvd,POINT (-13704001.270 5473411.225)
2,hub_3830,10th & Olive SE,POINT (-13702770.365 5472975.425)
3,hub_3831,16th & Willamette,POINT (-13702642.423 5471973.384)
4,hub_3832,13th & Pearl @ Pearl side,POINT (-13702278.750 5472449.473)


In [25]:
"Monroe St & Blair Blvd" in stations.name.values

True

In [40]:
tripLocations = [x for x in trips.Location.unique() if x != 'nan']

In [42]:
tripLocSelected = []

In [43]:
Loc2Review = []

In [44]:
# check if the trip location names are matched with the station names
for loc in tripLocations:
    if loc in stations.name.values:
        tripLocSelected.append(loc)
    else:
        print(loc)
        Loc2Review.append(loc)

nan
Carson Hall (virtual hub)
Court Cafe (virtual hub)
Event Rack 1 (Virtual Hub)
Millrace Garage


In [49]:
trips_selected = trips[trips.Location.isin(tripLocSelected)]

In [51]:
originCounts = trips_selected[trips_selected.OriginDestination == 'Origin'][['Location', 'RouteID', 'UserID']].groupby('Location').agg({"RouteID": lambda x: x.nunique(), "UserID": lambda x: x.nunique()})

In [52]:
originCounts.head()

,RouteID,UserID
Location,,
10th & Olive NE,198,109
10th & Olive SE,6334,1369
10th & Pearl,2082,735
12th & Willamette,12691,1885
13th & Agate (Virtual Hub),507,205


In [55]:
len(trips_selected.UserID.unique())

21761

In [54]:
trips_selected.shape[0]

565524

In [63]:
data = {'name':originCounts.index.values,
        'ntrips':originCounts.RouteID.values,
        'nbikers':originCounts.UserID.values}

In [64]:
df = pd.DataFrame(data)

In [66]:
origin_df = stations.merge(df, how='inner', on='name')

In [67]:
origin_df.head()

,station id,name,geometry,ntrips,nbikers
0,hub_3828,Eugene Family YMCA,POINT (-13701591.712 5471253.204),9618,1231
1,hub_3829,Monroe St & Blair Blvd,POINT (-13704001.270 5473411.225),2080,593
2,hub_3830,10th & Olive SE,POINT (-13702770.365 5472975.425),6334,1369
3,hub_3831,16th & Willamette,POINT (-13702642.423 5471973.384),4212,877
4,hub_3832,13th & Pearl @ Pearl side,POINT (-13702278.750 5472449.473),5013,1471


In [69]:
origin_df.to_file(r'T:\DCProjects\StoryMap\BikeCounting\BikeShare\Data\OriginCounts.shp')

In [70]:
destinationCounts = trips_selected[trips_selected.OriginDestination == 'Destination'][['Location', 'RouteID', 'UserID']].groupby('Location').agg({"RouteID": lambda x: x.nunique(), "UserID": lambda x: x.nunique()})